In [ ]:
import cv2 
import numpy as np

from keras.models import model_from_json

emotion_dict={0: "Angry", 1: "Disgust", 2: "Fear", 3: "Happy", 
                    4: "Neutral", 5: "Sad", 6: "Surprise"}

#Load json and create model
json_file=open("C:\\Users\\Lenovobhavya\\Documents\\Project_PBL\\data\\emotion_model_new.json", 'r')
loaded_model_json=json_file.read()
json_file.close ()
VGG16_emotion=model_from_json(loaded_model_json)
#Load weights into new model
VGG16_emotion.load_weights("C:\\Users\\Lenovobhavya\\Documents\\Project_PBL\\data\\emotion_model_new.h5")
print("loaded model from disk")
#start the webcam feed
cap=cv2.VideoCapture ("E:\\Final_year Project\\Training Videos\\SURPRISED\\surprised 7.mp4")
while True:
    #Find haar cascade to draw bounding box around face
    ret, frame=cap.read ()
    frame=cv2.resize(frame, (400,400))
    if not ret:
        break
    face_detector=cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    gray_frame=cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    
    #detect faces available in video
    num_faces=face_detector.detectMultiScale(gray_frame, scaleFactor=1.32, minNeighbors=5)
    #take each face available in the video and Preprocess it
    for (x, y, w, h) in num_faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0),2)
        roi_gray_frame=gray_frame [y:y+w, x:x+h]
        img_pix= np.expand_dims (np.expand_dims(cv2.resize(roi_gray_frame, (48, 48)), -1), 0)        

        
        #predict the emotions
        prediction=VGG16_emotion.predict(img_pix)
        maxindex = int(np.argmax (prediction))
        cv2. putText(frame, emotion_dict[maxindex], (x+5, y-20), cv2. FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)            
    #resized_img = cv2.resize(frame,(1000,700))
    #cv2.imshow('Face Sentiment Recognition', resized_img)
    cv2.imshow('Emotion Detection', frame)
    
    if cv2.waitKey(10) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()